In [ ]:
!pip install conllu
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3105/ud-treebanks-v2.5.tgz
!wget https://www.ida.liu.se/~TDDE09/commons/projectivize.py
!tar zxf ud-treebanks-v2.5.tgz

import os
import torch
import projectivize
import torch.nn as nn
from io import open
from conllu import parse_incr
import copy
import operator
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import subprocess

--2021-03-10 14:36:02--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3105/ud-treebanks-v2.5.tgz
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 355216681 (339M) [application/x-gzip]
Saving to: ‘ud-treebanks-v2.5.tgz’

ud-treebanks-v2.5.t 100%[===================>] 338.76M  25.1MB/s    in 15s     

2021-03-10 14:36:18 (23.2 MB/s) - ‘ud-treebanks-v2.5.tgz’ saved [355216681/355216681]

--2021-03-10 14:36:19--  https://www.ida.liu.se/~TDDE09/commons/projectivize.py
Resolving www.ida.liu.se (www.ida.liu.se)... 130.236.57.103, 2001:6b0:17:2004::57:103
Connecting to www.ida.liu.se (www.ida.liu.se)|130.236.57.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2878 (2.8K)
Saving to: ‘projectivize.py’

projectivize.py     100%[===================>]   2.81K  --.-KB/s    in 0s      

20

In [ ]:
 device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 print(device)

cpu


In [ ]:
#polish train data = UD_Polish-PDB/pl_pdb-ud-train.conllu
#english train data = UD_English-GUM/en_gum-ud-train.conllu

#arabic train data = UD_Arabic-PADT/ar_padt-ud-train.conllu
#arabic dev data = UD_Arabic-PADT/ar_padt-ud-dev.conllu

def datafiles(filename):
  data_file = open(filename, "r", encoding="utf-8")
  ud_files = []
  for tokenlist in parse_incr(data_file):
    ud_files.append(tokenlist)
  data = []
  for sentence in ud_files:
    current_sentence = []
    for token in sentence:
      current_token = []
      current_token.append(token['form'])
      current_token.append(token['upostag'])
      current_sentence.append(tuple(current_token))
    data.append(current_sentence)
  return data

#Arabic
arabic_train_data = datafiles("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-train.conllu")
arabic_dev_data = datafiles("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-dev.conllu")
#English
english_train_data = datafiles("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-train.conllu")
english_dev_data = datafiles("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-dev.conllu")

In [ ]:
print("Data Examples")
print(english_train_data[0])
print(arabic_train_data[0])
#We are not really using this dataset so move on haha

Data Examples
[('Aesthetic', 'ADJ'), ('Appreciation', 'NOUN'), ('and', 'CCONJ'), ('Spanish', 'ADJ'), ('Art', 'NOUN'), (':', 'PUNCT')]
[('برلين', 'X'), ('ترفض', 'VERB'), ('حصول', 'NOUN'), ('شركة', 'NOUN'), ('اميركية', 'ADJ'), ('على', 'ADP'), ('رخصة', 'NOUN'), ('تصنيع', 'NOUN'), ('دبابة', 'NOUN'), ('"', 'PUNCT'), ('ليوبارد', 'X'), ('"', 'PUNCT'), ('الالمانية', 'ADJ')]


In [ ]:
#Dataset Class
class Dataset():

    ROOT = ('<root>', '<root>', 0)  # Pseudo-root

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            tmp = [Dataset.ROOT]
            for line in lines:
                if not line.startswith('#'):  # Skip lines with comments
                    line = line.rstrip()
                    if line:
                        columns = line.split('\t')
                        if columns[0].isdigit():  # Skip range tokens
                            tmp.append((columns[1], columns[3], int(columns[6])))
                    else:
                        yield tmp
                        tmp = [Dataset.ROOT]

In [ ]:
dep_train_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-train.conllu")
dep_dev_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-dev.conllu")
#Example sentence list
print(list(dep_train_english_data)[0])

[('<root>', '<root>', 0), ('Aesthetic', 'ADJ', 2), ('Appreciation', 'NOUN', 0), ('and', 'CCONJ', 5), ('Spanish', 'ADJ', 5), ('Art', 'NOUN', 2), (':', 'PUNCT', 2)]


In [ ]:
PAD = '<pad>'
UNK = '<unk>'
def make_vocabs_head_tagger(gold_data):
    words = {PAD:0, UNK:1}
    tags =  {PAD:0}

    word_i = 2
    tag_i = 1

    for sentence in gold_data:
      for word,tag, _ in sentence:
        if tag not in tags:
          tags[tag] = tag_i
          tag_i +=1
        if word not in words:
          words[word] = word_i
          word_i +=1

    return words, tags

In [ ]:
word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_english_data)

In [ ]:
#Arc Parser and oracle move implementation
class Parser(object):

    def predict(self, words, tags):
        raise NotImplementedError

class ArcStandardParser(Parser):

    MOVES = tuple(range(3))

    SH, LA, RA = MOVES  # Parser moves are specified as integers.

    @staticmethod
    def initial_config(num_words):
        # Replaced the next line with our own code
        i = 0
        stack = []
        heads = [0]*num_words
        return (i,stack,heads)

    @staticmethod
    def valid_moves(config):
        # Replaced the next line with our own code
        (SH, LA, RA) = ArcStandardParser.MOVES
        (i, stack, heads) = config[:]
        valid_moves = []
        if i < len(heads):
            valid_moves.append(SH)
        if len(stack) > 1:
            valid_moves.append(LA)
            valid_moves.append(RA)
        return valid_moves

    @staticmethod
    def next_config(config, move):
        # Replaced the next line with our own code
        (SH, LA, RA) = ArcStandardParser.MOVES
        i = config[0]
        stack = config[1][:]
        heads = config[2][:]
        if move == SH:
            stack.append(i)
            i += 1
        elif move == LA:
            head = stack[-1]
            word = stack.pop(-2)
            heads[word] = head
        elif move == RA:
            head = stack[-2]
            word = stack.pop(-1)
            heads[word] = head
        return (i,stack,heads)

    @staticmethod
    def is_final_config(config):
        # Replaced the next line with our own code
        return len(ArcStandardParser.valid_moves(config)) < 1

#new oracle 2
def has_no_remaining_arcs(word, config, gold_heads):
    return config[2].count(word) == gold_heads.count(word)

def oracle_moves(gold_heads):
    parser = ArcStandardParser()
    (SH, LA, RA) = parser.MOVES
    config = parser.initial_config(len(gold_heads))
    oracle_moves = []
    while not parser.is_final_config(config):
        valid_moves = parser.valid_moves(config)
        if LA in valid_moves: #LA condition is the same as RA
            i_la, stack_la, heads_la = parser.next_config(config, LA) #i_la, stack_la, heads_la=config_LA
            i_ra, stack_ra, heads_ra = parser.next_config(config, RA) #i_ra, stack_ra, heads_ra=config_RA
            secmost = config[1][-2]
            topmost = config[1][-1]
            if(gold_heads[secmost] == heads_la[secmost] and has_no_remaining_arcs(secmost, config, gold_heads)):
            #if(gold_heads[secmost] == heads_la[secmost]):   #This line gets same results
                yield (config, LA)
                config = (i_la, stack_la, heads_la)
                continue

            elif(gold_heads[topmost] == heads_ra[topmost] and has_no_remaining_arcs(topmost, config, gold_heads)):
                yield (config, RA)
                config = (i_ra, stack_ra, heads_ra)
                continue

        if SH in valid_moves: #Same as using else instead os if sh...
            yield (config, SH)
            config = parser.next_config(config, SH)

        else: #?
            print("?")
            break

In [ ]:
#biLSTM concatenate words and tags (xi=e(wi)◦e(pi)) so there is no need to add ps2,1,0 here
def get_feats(config, word_ids):
  b, s, h = config

  vs2, vs1, vs0 = [0,0,0]
  b0 = 0
  if(b!=0):
    b0 = b - 1
  if(len(s) == 1):
    vs0 = max(s[-1] - 1 , 0) #if s[-1] == 0 (-1) and s[-1] == 1 (0) => vs0 = 0 ('<root>')

  elif(len(s) == 2):
    vs0 = max(s[-1] - 1 , 0)
    vs1 = max(s[-2] - 1 , 0)

  elif(len(s) >= 3):
    vs0 = max(s[-1] - 1 , 0)
    vs1 = max(s[-2] - 1 , 0)
    vs2 = max(s[-3] - 1 , 0)

  feats_from_config = [vs2, vs1, vs0, b0] #Index of vs2, vs1, vs0, bs0 from config
  feats_torch = torch.LongTensor(feats_from_config) #torch.Size([1, 4])
  return torch.LongTensor(feats_torch)

In [ ]:
def training_example(vocab_words, vocab_tags, gold_data):
    for sentence in gold_data: #[('<root>', '<root>', 0), ('Aesthetic', 'ADJ', 2), ('Appreciation', 'NOUN', 0), ('and', 'CCONJ', 5), ('Spanish', 'ADJ', 5), ('Art', 'NOUN', 2), (':', 'PUNCT', 2)]
        word_ids, tag_ids, gold_heads = [], [], []
        moves = []
        idxs = []
        #create id-lists for the current sentence
        for tup in (sentence):
            word_ids.append(vocab_words[tup[0]]) # Ids of ['<root>', 'Aesthetic'... ]
            tag_ids.append(vocab_tags[tup[1]]) # Ids of ['<root>', 'ADJ'... ]
            gold_heads.append(tup[2]) #[0, 2, ...]

        for config, move in oracle_moves(gold_heads): #for each sentence we get oracle moves
            idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
            idxs.append(idx_feats)
            moves.append(move)

        yield torch.LongTensor(word_ids).unsqueeze(0), torch.LongTensor(tag_ids).unsqueeze(0), torch.stack(idxs), torch.LongTensor(moves)

In [ ]:
#Example of one sentence in training example function
w_ids, t_ids, f_idx, mov = next(training_example(word_vocab, tag_vocab, dep_train_english_data))
print("word ids: ",w_ids)
print("tag ids: ",t_ids)
print("mlp features indexs: " ,f_idx.shape)
print("force trinning moves: ", mov.shape)

word ids:  tensor([[2, 3, 4, 5, 6, 7, 8]])
tag ids:  tensor([[1, 2, 3, 4, 2, 3, 5]])
mlp features indexs:  torch.Size([13, 4])
force trinning moves:  torch.Size([13])


In [ ]:
#"Encoder" Implementation
class BiLSTM(nn.Module):
    def __init__(self, num_words, num_pos, word_emb_dim = 100, pos_emb_dim=25, hidden_dim = 125, n_layers=2):
        super().__init__()
        # for the BiLSTM
        self.ew = nn.Embedding(num_words, word_emb_dim)
        self.ep = nn.Embedding(num_pos, pos_emb_dim)
        self.bi_lstm = nn.LSTM(word_emb_dim + pos_emb_dim, hidden_dim, num_layers = n_layers, batch_first=True, bidirectional = True)

    def forward(self, word, pos):
        ew = self.ew(word)            # Shape: [Batch_size, n_words, word_emb_dim]
        ep = self.ep(pos)             # Shape: [Batch_size, n_words, pos_emb_dim]
        x = torch.cat((ew, ep), 2)    # Shape: [Batch_size, n_words, word_emb_dim + pos_emb_dim]
        output, _ = self.bi_lstm(x)   # Shape: [Batch_size, n_words, 2*(word_emb_dim + pos_emb_dim)]

        return output

In [ ]:
#w_ids, t_ids, g_heads sentence example
model_lstm = BiLSTM(len(word_vocab), len(tag_vocab))
test_output = model_lstm.forward(w_ids, t_ids)
print("Shape BiLSTM feats: ",test_output.shape)

Shape BiLSTM feats:  torch.Size([1, 7, 250])


In [ ]:
class MLP(nn.Module):
    def __init__(self, word_emb_dim = 100, pos_emb_dim=25, hidden_unit = 100, output_dim = 3):
        super().__init__()
        self.hidden = nn.Linear(4*2*(word_emb_dim + pos_emb_dim), hidden_unit)
        self.relu = nn.ReLU() # try relu
        self.output = nn.Linear(hidden_unit, output_dim)

    def forward(self, features):
        hn = self.hidden(features)
        out = self.output(self.relu(hn))

        return out

In [ ]:
#Example of MLP
#feats_from_config is a list of list of index bc each list represent a sentence, in this case batch_size = 1 so [[ words ]]
#this is for no batch (we are using this one)
print("Example for no batch | biLSTM output shape: ", test_output.shape)
feats_from_config = [0, 3, 4, 5] #Index of vs2, vs1, vs0, bs0 from config
feats_torch = torch.LongTensor(feats_from_config) #torch.Size([1, 4])
test_input_mlp = test_output.index_select(1, feats_torch)
print("Shape input for mlp: " ,test_input_mlp.shape)
test_input_mlp = test_input_mlp.view(-1,test_input_mlp.size(1)*test_input_mlp.size(2))
print("Concat embeds shape: ", test_input_mlp.shape)

print("#################################")

#Batch size > 1 case
test_output2 = torch.cat((test_output,test_output), 0)
print("Example for batch feats | biLSTM output shape: ", test_output2.shape)
feats_from_config2 = [[0, 3, 4, 5], [0, 3, 4, 5]]
feats_torch2 = torch.LongTensor(feats_from_config2)
indexs = feats_torch2.unsqueeze(2).expand(feats_torch2.size(0), feats_torch2.size(1), test_output2.size(2))
test_input_mlp2 = torch.gather(test_output2, 1, indexs)
print("Shape input for mlp: " ,test_input_mlp2.shape)
test_input_mlp2 = test_input_mlp2.view(-1,test_input_mlp2.size(1)*test_input_mlp2.size(2))
print("Concat embeds shape: ",test_input_mlp2.shape)

print("################################")
#Case 3 (list of list of index for one sentence (batch=1))
#Every list of index represent the setting for a move, in this case 2
test_output3 = test_output
print("Example for no batch | biLSTM output shape: ", test_output3.shape)
feats_from_config3 = [[0, 3, 4, 5], [0, 3, 4, 5]]
feats_torch3 = torch.LongTensor(feats_from_config3) #torch.Size([2, 4])
test_input_mlp3 = [test_output3.index_select(1, idxs) for idxs in feats_torch3]
test_input_mlp3 = torch.stack(test_input_mlp3)
test_input_mlp3 = test_input_mlp3.squeeze(1)
print("Shape input for mlp: " ,test_input_mlp3.shape)
test_input_mlp3 = test_input_mlp3.view(-1,test_input_mlp3.size(1)*test_input_mlp3.size(2))
print("Concat embeds shape: ", test_input_mlp3.shape)

Example for no batch | biLSTM output shape:  torch.Size([1, 7, 250])
Shape input for mlp:  torch.Size([1, 4, 250])
Concat embeds shape:  torch.Size([1, 1000])
#################################
Example for batch feats | biLSTM output shape:  torch.Size([2, 7, 250])
Shape input for mlp:  torch.Size([2, 4, 250])
Concat embeds shape:  torch.Size([2, 1000])
################################
Example for no batch | biLSTM output shape:  torch.Size([1, 7, 250])
Shape input for mlp:  torch.Size([2, 4, 250])
Concat embeds shape:  torch.Size([2, 1000])


In [ ]:
# example data features = test_input_mlp2
model_MLP = MLP()
test_output_mlp = model_MLP.forward(test_input_mlp)
print("Shape BiLSTM feats: ",test_output_mlp.shape)
test_output_mlp2 = model_MLP.forward(test_input_mlp2)
print("Shape BiLSTM feats: ",test_output_mlp2.shape)

Shape BiLSTM feats:  torch.Size([1, 3])
Shape BiLSTM feats:  torch.Size([2, 3])


In [ ]:
class Blackbox(nn.Module): #need to change name lol haha
    def __init__(self, vocab_words, vocab_tags, num_words, num_pos):
        super().__init__()

        self.model_lstm = BiLSTM(num_words, num_pos)
        self.model_MLP = MLP()
        self.vocab_words = vocab_words
        self.vocab_tags = vocab_tags

    def forward(self, word_ids, pos_ids, idxs_feats):
        lstm_output = self.model_lstm.forward(word_ids, pos_ids) #biLSTM output shape:  torch.Size([1, 7, 250])
        #input_mlp = lstm_output.index_select(1, idxs_feats) #Shape input for mlp:  torch.Size([1, 4, 250])
        input_mlp = [lstm_output.index_select(1, idxs) for idxs in idxs_feats] #len(n) = moves
        input_mlp = torch.stack(input_mlp) #to tensor
        input_mlp = input_mlp.squeeze(1) #shape tesnro ([n, 4, 250]) where n: moves
        input_mlp = input_mlp.view(-1,input_mlp.size(1)*input_mlp.size(2)) #Concat embeds shape:  torch.Size([n, 1000])
        output_mlp = self.model_MLP.forward(input_mlp) #torch.Size([n, 3])

        return output_mlp

    def predict(self, words, tags):
        for idx, (w, t) in enumerate(zip(words,tags)): #words string to words ids
            if w in self.vocab_words:
                words[idx] = self.vocab_words[w]
            else:
                words[idx] = self.vocab_words['<unk>']
            if t in self.vocab_tags:
                tags[idx] = self.vocab_tags[t]
            else:
                tags[idx] = self.vocab_tags['<pad>']

        config = ArcStandardParser.initial_config(len(words))
        while not ArcStandardParser.is_final_config(config):
            idx_feats = get_feats(config, words) #take features #idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
            valid_moves = ArcStandardParser.valid_moves(config)

            with torch.no_grad():
                output_move = torch.argmax(self.forward(words, tags, idx_feats)) #output_mlp = self.forward(words, tags, idx_feats)
            if(output_move in valid_moves):
                config = parser.next_config(config, output_move)
            else:
                config = parser.next_config(config, valid_moves[-1])

        return config[2]

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import tqdm
torch.autograd.set_detect_anomaly(True)

def train_model(train_data, n_epochs = 1, batch_size = 0, lr=1e-2):
    word_vocab, tag_vocab = make_vocabs_head_tagger(train_data)
    model = Blackbox(word_vocab, tag_vocab, len(word_vocab), len(tag_vocab)) #  model_lstm = BiLSTM(len(word_vocab), len(tag_vocab)) |  model_MLP = MLP()
    #model.to(device)
    # Initialize the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    dev_losses = []
    dev_accuracies = []
    info = {'dev loss': 0, 'dev acc': 0}
    x = 0
    with tqdm.tqdm(total=3753) as pbar: #len(dep_train_english_data) = 3753
        for t in range(n_epochs):
            model.train()
            # start training
            running_loss = 0
            t_data = training_example(word_vocab, tag_vocab, train_data)

            for w_ids, t_ids, f_idx, move in t_data: #This represet a sentence w_ids, t_ids, f_idx, mov
                pbar.set_description(f'Sentence {x+1}')
                optimizer.zero_grad()
                output_mlp = model.forward(w_ids, t_ids, f_idx)
                loss = F.cross_entropy(output_mlp, move)
                loss.backward() #retain_graph=True
                optimizer.step()

                if(x%250 == 0):
                    print("Loss: "+str(loss.item()))

                x += 1
                pbar.update(1)
                running_loss += loss.item()

                train_losses.append(running_loss / x)

                #if(x == 500): #Train only 500 sentences
                #  break

    return model

In [ ]:
#model_BB = train_model(dep_train_english_data) #UAS: 0.6553


Sentence 3:   0%|          | 2/3753 [00:00<10:33,  5.92it/s]

Loss: 1.104910135269165


Sentence 7:   0%|          | 6/3753 [00:01<13:53,  4.49it/s]

?


Sentence 8:   0%|          | 7/3753 [00:01<14:29,  4.31it/s]

...

Sentence 3750: 100%|█████████▉| 3749/3753 [25:52<00:01,  2.81it/s]

?


Sentence 3752: 100%|█████████▉| 3751/3753 [25:53<00:00,  2.42it/s]

Loss: 0.14921803772449493


Sentence 3753: 100%|██████████| 3753/3753 [25:54<00:00,  2.41it/s]


In [ ]:
print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_english_data)))

0.6553


In [ ]:
def uas_tst(modelo, vocab_words, vocab_tags, gold_data):
    correct, total = 0, 0
    for sentence in gold_data:
      words,tags, heads = [], [], []

      for word, tag, head in (sentence):
        words.append(word)
        tags.append(tag)
        heads.append(head)
      #print(words)
      preds = predict_tst(modelo, vocab_words, vocab_tags, words, tags)
      #print(preds)

      for i in range(1,len(preds)):
        if heads[i] == preds[i]:
          correct +=1
        total +=1

    return correct/total

def predict_tst(modelo, vocab_words, vocab_tags, words, tags):
    for idx, (w, t) in enumerate(zip(words,tags)): #words string to words ids
      if w in vocab_words:
        words[idx] = vocab_words[w]
      else:
        words[idx] = vocab_words['<unk>']
      if t in vocab_tags:
        tags[idx] =vocab_tags[t]
      else:
        tags[idx] = vocab_tags['<pad>']


    words = torch.LongTensor(words).unsqueeze(0)
    tags = torch.LongTensor(tags).unsqueeze(0)

    parser = ArcStandardParser()
    (SH, LA, RA) = parser.MOVES
    config = parser.initial_config(words.shape[1])
    while not parser.is_final_config(config):
      idx_feats = get_feats(config, words) #take features #idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
      #idx_feats = get_feats(config, words) #long_tensor([vs0, vs1, vs2, b0]))
      idx_feats = idx_feats.unsqueeze(0)
      valid_moves = parser.valid_moves(config)

      with torch.no_grad():
        output_move = torch.argmax(modelo.forward(words, tags, idx_feats)) #output_mlp = self.forward(words, tags, idx_feats)
      if(output_move in valid_moves):
        config = parser.next_config(config, output_move)
      else:
        config = parser.next_config(config, valid_moves[-1])

    return config[2]